In [15]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import Ridge
from scipy.sparse import hstack

#1. Загрузите данные об описаниях вакансий и соответствующих годовых зарплатах из файла salary-train.csv 
data = pd.read_csv('salary-train.csv')

#2. Проведите предобработку:
def transform(text):
    #Приведите тексты к нижнему регистру (text.lower()).
    text = text.map(lambda x: x.lower())
    #Замените все, кроме букв и цифр, на пробелы — это облегчит дальнейшее разделение текста на слова. 
    text = text.replace('[^a-zA-Z0-9]',regex = True)
    return text
#Примените TfidfVectorizer для преобразования текстов в векторы признаков. Оставьте только те слова,
#которые встречаются хотя бы в 5 объектах (параметр min_df у TfidfVectorizer).
vectorizer = TfidfVectorizer(min_df = 5)
x_train_data = vectorizer.fit_transform(transform(data['FullDescription']))

#Замените пропуски в столбцах LocationNormalized и ContractTime на специальную строку 'nan'.
data['LocationNormalized'].fillna('nan', inplace = True)
data['ContractTime'].fillna('nan', inplace = True)

#Примените DictVectorizer для получения one-hot-кодирования признаков LocationNormalized и ContractTime.
enc = DictVectorizer()
x_train_categ = enc.fit_transform(data[['LocationNormalized', 'ContractTime']].to_dict('records'))

#Объедините все полученные признаки в одну матрицу "объекты-признаки". Для объединения их столбцов 
#нужно воспользоваться функцией scipy.sparse.hstack.

x_train = hstack([x_train_data,x_train_categ])

#3. Обучите гребневую регрессию с параметрами alpha=1 и random_state=241. Целевая переменная 
#записана в столбце SalaryNormalized.

y_train = data['SalaryNormalized']
model = Ridge(alpha = 1, random_state = 241)
model.fit(x_train, y_train)

#4. Постройте прогнозы для двух примеров из файла salary-test-mini.csv. Значения полученных прогнозов 
#являются ответом на задание. Укажите их через пробел.

test = pd.read_csv('salary-test-mini.csv')
x_test_text = vectorizer.transform(transform(test['FullDescription']))
x_test_categ = enc.transform(test[['LocationNormalized', 'ContractTime']].to_dict('records'))
x_test = hstack([x_test_text, x_test_categ])

y_test = model.predict(x_test)
print(round(y_test[0],2), round(y_test[1],2))



56547.75 37200.15
